In [146]:
from synthcity.metrics.eval import Metrics
import pandas as pd
def synthcity_evaluate(real_path,
                       syn_path,
                       result_name=None,
                       real_size = None,
                       syn_size = None):
    real_data = pd.read_csv(real_path)
    syn_data = pd.read_csv(syn_path)
    
    if real_size and real_size < len(real_data):
        real_data = real_data.sample(n=real_size, random_state=42)  # Adjust random_state as desired for reproducibility
    if syn_size and syn_size < len(syn_data):
        syn_data = syn_data.sample(n=syn_size, random_state=42)
    
    synthcity_result = Metrics.evaluate(real_data, syn_data)
    if result_name:
        synthcity_result.to_csv(result_name)
    # synthcity_result.columns = "metrics" + synthcity_result.columns
    # synthcity_result.columns = synthcity_result.columns.insert(0, "metrics")
    return synthcity_result
    
def compare(real_file_name, no_water_mark_file_name, with_water_mark_file_name, size):
    samples_dir = "/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/samples/"
    real_path = samples_dir + real_file_name
    synth_no_watermark_path = samples_dir + no_water_mark_file_name
    synth_with_watermark_path = samples_dir + with_water_mark_file_name
    
    
    real_df = synthcity_evaluate(real_path, real_path, real_size=size, syn_size=size)[["min"]]
    no_water_mark_df = synthcity_evaluate(real_path, synth_no_watermark_path, real_size=size, syn_size=size)[["min"]]
    with_watermark_df = synthcity_evaluate(real_path, synth_with_watermark_path, real_size=size, syn_size=size)[["min", "direction"]]
    merge_result = real_df.join(no_water_mark_df, lsuffix='_real', rsuffix='_no_watermark')
    merge_result_final = merge_result.join(with_watermark_df)
    
    merge_result_final.columns = ["Real Data", "No-Watermark", "With-Watermark", "Direction"]
    merge_result_final = merge_result_final.round(2)
    
    interested_metrics = [
     "sanity.nearest_syn_neighbor_distance.mean", 
        "stats.ks_test.marginal",
        "performance.linear_model.gt",
        "performance.xgb.gt",
        "performance.mlp.gt"
    ]

    # merge_result_final = merge_result_final.loc[merge_result_final["Metrics"].isin(interested_metrics)]
    return merge_result_final

compare(
    "california.csv",
    "california_100_non-watermark.csv",
    "california_100_with-watermark_gamma-0.25_delta-2.0.csv",
    size=100
)



,Real Data,No-Watermark,With-Watermark,Direction
sanity.data_mismatch.score,0.00,0.00,0.00,minimize
sanity.common_rows_proportion.score,1.00,0.00,0.00,minimize
sanity.nearest_syn_neighbor_distance.mean,0.00,0.04,0.05,minimize
sanity.close_values_probability.score,1.00,0.96,0.96,maximize
sanity.distant_values_probability.score,0.00,0.01,0.01,minimize
stats.jensenshannon_dist.marginal,0.00,0.02,0.03,minimize
stats.chi_squared_test.marginal,1.00,0.11,0.22,maximize
stats.inv_kl_divergence.marginal,1.00,0.54,0.57,maximize
stats.ks_test.marginal,1.00,0.86,0.72,maximize
stats.max_mean_discrepancy.joint,0.00,0.02,0.02,minimize
